In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
df_edges = pd.read_csv('./data/graph_edges_with_directionality - Kopie.csv')

G = nx.DiGraph()
for orig,desti in zip(df_edges['origin_place_id'], df_edges['destination_place_id']):
    G.add_edge(orig, desti)
nodes = list(G.nodes())

In [3]:
train_data = np.load("./DCRNN/data/OUR/train.npz")
test_data = np.load("./DCRNN/data/OUR/test.npz")

In [4]:
for cat in train_data:
    print(cat)

x
y
x_offsets
y_offsets


In [6]:
train_data['x'].shape, train_data['y'].shape

((18262, 12, 270, 2), (18262, 12, 270, 2))

In [7]:
train_data['x'][:,0,0,0]

array([0.11114049, 0.10701927, 0.09617724, ..., 0.21353762, 0.20221071,
       0.19852285])

In [8]:
train_data['x'][:,0,0,1]

array([0.20833333, 0.21180556, 0.21527778, ..., 0.56597222, 0.56944444,
       0.57291667])

In [9]:
def get_new_shape(data):
    new_shape = list(data.shape)
    new_shape[3] = 3
    return tuple(new_shape)

In [10]:
cleaner_train = [max(np.append(tx,ty)) == ty[-1] for tx,ty in zip(train_data['x'][:,:,0,1], train_data['y'][:,:,0,1])]
cleaner_test = [max(np.append(tx,ty)) == ty[-1] for tx,ty in zip(test_data['x'][:,:,0,1],test_data['y'][:,:,0,1])]

In [11]:
new_train_data_x = np.zeros(get_new_shape(train_data['x'][cleaner_train]))

In [12]:
new_test_data_x = np.zeros(get_new_shape(test_data['x'][cleaner_test]))

In [13]:
new_train_data_x[:,:,:,[0,2]] = train_data['x'][cleaner_train]
new_train_data_y = train_data['y'][cleaner_train]

new_test_data_x[:,:,:,[0,2]] = test_data['x'][cleaner_test]
new_test_data_y = test_data['y'][cleaner_test]

In [18]:
day_times = new_train_data_x[:,0,0,2][np.arange(181,362)]

In [19]:

for i in range(len(day_times)):
    print(i)
    days_ind_train = np.array([new_train_data_x[:,0,0,2] == day_times[i]])[0]
    new_train_data_x[days_ind_train,:,:,1] = np.mean(new_train_data_y[days_ind_train,:,:,0], axis = 0)

    days_ind_test = np.array([new_test_data_x[:,0,0,2] == day_times[i]])[0]
    new_test_data_x[days_ind_test,:,:,1] = np.mean(new_train_data_y[days_ind_train,:,:,0], axis = 0)

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [ ]:
np.savez_compressed(
    "train_hist_avg",
    x=new_train_data_x,
    y=new_train_data_y,
)

In [22]:
np.savez_compressed(
    "test_hist_avg",
    x=new_test_data_x,
    y=new_test_data_y,
)

In [ ]:
np.mean(new_train_data_x[:,:,:,0][new_train_data_x[:,0,0,1] == day_times[i]], axis = 0).shape